# Tools

This notebook contains all the tools that will be used by the agent.

In [2]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [ ]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('./data/cities_latlng.csv', sep=";")

# Add headers to our data frame
df.columns = [ 'id', 'country', 'city', 'latitude', 'longitude', 'altitude']

# Drop country column
df.drop('country', axis=1, inplace=True)

In [4]:
# TODO: Explore the loaded dataframe
df.head()

,id,city,latitude,longitude,altitude
0,2,Kandahar,31.610000,65.699997,1015
1,3,Mazar-e Sharif,36.706944,67.112221,369
2,4,Herat,34.340000,62.189999,927
3,5,Jalalabad,34.420000,70.449997,573
4,6,Konduz,36.720000,68.860001,394


In [7]:
df[df['city'] == 'Singapore']

,id,city,latitude,longitude,altitude
10566,10568,Singapore,1.29027,103.851959,164


In [14]:
# TODO: Add tool description

# Decorate the function with @tool
@tool
def get_latlng(city: str) -> any:
   """
   Get the latitude, longitude and altitude of a city 
   
   Args:
      city: the city name 
   
   Returns:
      any: dictionary with the following keys: city, latitude, longitude, altitude

   Example:
      result = get_latlag('tokyo')
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': float(r.iloc[0]['latitude']), 'longitude': float(r.iloc[0]['longitude']), 'altitude': float(r.iloc[0]['altitude']) }

In [15]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')

{'city': 'tokyo',
 'latitude': 35.6895266,
 'longitude': 139.6916809,
 'altitude': 40.0}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [ ]:
# TODO: Add tool description
@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """ 
   Get the temperate of a location given by its latitude and longitude
   Args:
      latitude: latitude of a location
      longitude: corresponding longitude of a location
   Returns:
      any: dictionary with the following keys: temperature_unit, temperature
   Example:
      result = get_temperature(35.6895266, 139.6916809)
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [17]:
# TODO: Test get_temperature method
get_temperature(35.6895266, 139.6916809)

{'temperature_unit': '°C', 'temperature': 19.9}

In [ ]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [ ]:
# TODO: Test the web search


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [23]:
# TODO: Add tool description
@tool
def query_album_track(query: str) -> any:
   """ 
   Perform SQL queries on the album_track table. Returns the result as an array of tuples. 
   The table name is album_track and has the follwing columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration column is in milliseconds

   Args:
      query: Valid SQL statement

   Returns:
      any: list of tuple. Each element of the tuple corresponds to the result of the query

   Example:
      results = query_album_track('select count(*) from album_track')
   """
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [22]:
# TODO: Test the query_album_track function
query_album_track('select title, track_name, artist_name from album_track where artist_name like "%son%"')

[('Chemical Wedding', 'King In Crimson', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Chemical Wedding', 'Bruce Dickinson'),
 ('Chemical Wedding', 'The Tower', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Killing Floor', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Book Of Thel', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Gates Of Urizen', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Jerusalem', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Trupets Of Jericho', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Machine Men', 'Bruce Dickinson'),
 ('Chemical Wedding', 'The Alchemist', 'Bruce Dickinson'),
 ('Chemical Wedding', 'Realword', 'Bruce Dickinson'),
 ('Out Of Time', 'Shiny Happy People', 'R.E.M. Feat. Kate Pearson'),
 ('Out Of Time', 'Me In Honey', 'R.E.M. Feat. Kate Pearson'),
 ('Out Of Time', 'Radio Song', 'R.E.M. Feat. Kate Pearson'),
 ('Out Of Time', 'Losing My Religion', 'R.E.M. Feat. Kate Pearson'),
 ('Out Of Time', 'Low', 'R.E.M. Feat. Kate Pearson'),
 ('Out Of Time', 'Near Wild Heaven', 'R.

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [ ]:
class SQLiteTool(Tool):

   # need to give the tool a name
   name = "artist_database"

   # description 
   description = """ 
   Perform SQL queries on the album_track table. Returns the result as an array of tuples. 
   The table name is album_track and has the follwing columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration column is in milliseconds
   """

   # arguments 
   inputs = {
      "query": {
         "type": "string", 
         "description": "Valid SQL statement"
      }
   }

   # Return type
   output_type = "any"

   def __init__(self, db_file):
      self.db_file = db_file 
      self.setup()

   def setup(self):
      super().setup()
      self.conn = sqlite3.connect(self.db_file)

   def forward(self, query: str) -> any:
      try:
         cursor = self.conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      except Exception as e:
         print(f'Query exception: {e}')  